## Transfer Resting State Eyetracking Files from USB to Backup Device (.ps1)

VERSION: `Structural_Eyetrack_Auto_Push`

### Advanced Explanation

The script is designed to selectively transfer directories from a source to a destination path based on user input. It first defines the source (`$source`) and destination (`$destination`) paths. Using `Get-ChildItem`, it retrieves subdirectories in `$source` that were last written to within the last 10 hours.

In the loop over each directory (`$dir`) in `$rawdata`, the script calculates the size in megabytes of each directory by summing the size of all files within (`Get-ChildItem -Recurse -File` with `Measure-Object`). It then prompts the user with details of the directory (name, size, creation time), asking if they wish to transfer it to the destination (`$destination`). If the user inputs "Y", the script uses `Copy-Item -Recurse` to copy the entire directory to the destination, confirming each copy with an output message.

### User-Friendly Explanation

This script is a tool for backing up folders from one part of your computer to a USB drive. It looks at specific folders that have been updated in the last 10 hours. For each of these folders, it shows you the folder's name, size, and when it was created, and asks if you want to back it up to your USB.

If you choose to back up a folder, it copies the entire folder to the USB. The script tells you once it has successfully copied a folder. This way, you can selectively back up recent folders to your USB, ensuring you have the latest data saved.

### Oversimplified Explantion

The script is designed so that files created after the last sync will be copied over from the USB to the Backupdata folder. (This means you could fix a folder name and the old version won't be copied over again). Still make sure to check if the session folders pushed are the ones expected.  

In [ ]:
$source = "E:\RestingState_DAY1" 
$destination = "C:\Backupdata\RestingState_DAY1" 
$logFile = "C:\Backupdata\RestingState_DAY1\DAY1_USB_Log.txt" 

if (-not (Test-Path -Path $logFile)) {
    New-Item -Path $logFile -ItemType File
    "Log Created: $(Get-Date)" | Out-File -FilePath $logFile
}

$lastSyncTime = (Get-Content -Path $logFile -Tail 1) -replace "Last Sync:", ""
if ([string]::IsNullOrWhiteSpace($lastSyncTime)) {
    $lastSyncTime = [datetime]::MinValue
} else {
    $lastSyncTime = [datetime]::Parse($lastSyncTime)
}

$newData = Get-ChildItem -Path $source -Directory | Where-Object { $_.CreationTime -gt $lastSyncTime }

if ($newData.Count -eq 0) {
    Write-Output "No new session folders since last synced."
} else {
    foreach ($dir in $newData) {
        $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
        $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
        $creationTime = $dir.CreationTime

        $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($dirSizeMB) MB, Created: $($creationTime)]
         from $source ? [Y/N]"

        if ($userInput -eq "Y") {
            $destPath = Join-Path -Path $destination -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to the computer"
        }
    }
}

"Last Sync: $(Get-Date)" | Out-File -FilePath $logFile -Append
